In [1]:
import numpy as np
import statsmodels.stats.api as sms
from scipy.stats import t as t
import math
from itertools import combinations
import threading
#from tqdm import tqdm

In [2]:
from src.search import search
from src.bootstrap import bootstrap_ci
from src.perm_test import find_intervals, ttest_ind, pval
from src.utils import partition

---

In [3]:
np.random.seed(123)
n1, n2 = 2, 3
gamma1 = (2, 5)  # shape k, scale theta
gamma2 = (4, 3)
x1 = np.random.gamma(gamma1[0], gamma1[1], n1)
x2 = np.random.gamma(gamma2[0], gamma2[1], n2)
print(x1)
print(x2)

[ 3.10640204 16.57130493]
[ 7.99973308 23.47536422  2.11936196]


In [4]:
partitions = partition(n1, n2)
partitions

array([[0, 1, 2, 3, 4],
       [0, 2, 1, 3, 4],
       [0, 3, 1, 2, 4],
       [0, 4, 1, 2, 3],
       [1, 2, 0, 3, 4],
       [1, 3, 0, 2, 4],
       [1, 4, 0, 2, 3],
       [2, 3, 0, 1, 4],
       [2, 4, 0, 1, 3],
       [3, 4, 0, 1, 2]])

In [ ]:
find_intervals(1, partitions, n1, n2, gamma1, gamma2,
               pooled=True, alternative="two-sided", save_intervals=True)

---

In [ ]:
thread_count = 16
threads = [0] * thread_count

remaining = 128
batch_size = remaining // thread_count

intervals = []
n_captured = 0

In [ ]:
%%time

for i in range(thread_count):
    n_samples = batch_size if i < thread_count-1 else remaining
    threads[i] = threading.Thread(target=passToThread, args=(n_samples,))
    threads[i].start()
    remaining -= n_samples
    
for thread in threads:
    thread.join()

In [ ]:
n_captured, len(intervals)

Multithreading allows us to compute one confidence interval in \~80 ms, even faster than it took to compute the one confidence interval above (\~360 ms).